해당 repository 및 code들은 wikidocs에서 제공하는 'Pytorch로 시작하는 딥 러닝 입문'을 참고하였음을 밝힙니다. 해당 자료를 바탕으로 숙지한 개념과 추가적인 저의 생각을 기록할 계획입니다. 출처는 다음 사이트와 같습니다.<br><br>
* https://wikidocs.net/book/2788

```python
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# 랜덤 시드 고정
torch.manual_seed(777)

# GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

learning_rate = 0.001
training_epochs = 15
batch_size = 100

mnist_train = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                          train=True, # True를 지정하면 훈련 데이터로 다운로드
                          transform=transforms.ToTensor(), # 텐서로 변환
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/', # 다운로드 경로 지정
                         train=False, # False를 지정하면 테스트 데이터로 다운로드
                         transform=transforms.ToTensor(), # 텐서로 변환
                         download=True)

data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)
            
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # 첫번째층
        # ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 두번째층
        # ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        # 전결합층 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        # 전결합층 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        return out

# CNN 모델 정의
model = CNN().to(device)

criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

total_batch = len(data_loader)
print('총 배치의 수 : {}'.format(total_batch))

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

# 학습을 진행하지 않을 것이므로 torch.no_grad()
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())
```

위의 코드가 Pytorch로 구현한 CNN based architecture이다. Keras와 비교했을때, 좀 더 구체적으로 코드를 짜야 한다는 느낌을 받았다. 즉, Keras API가 사용자에게 좀 더 많은 정보를 편리하게 제공하고 있다는 의미이기도 하다. 간단한 예를 들자면, Keras는 모델을 학습시킬 때 설정한 값에 따라 epoch과 학습 시간, 평가지표(metrics), loss 값 등이 표시된다. 반면 Pytorch는 이러한 것들을 print문을 통해 스스로 구현해야 한다(물론 이를 구현하는 다른 library가 존재할 수도 있겠다). 따라서 둘의 차이점을 잘 알아야 하는데, 특히 Pytorch에서 중요하게 봐야 할 부분들을 정리해두겠다.

### 1. device 설정

Pytorch는 변수별로 CPU와 GPU 중 처리 방법을 선택할 수 있다. 이를 위해 device를 설정하는데, 만약 GPU가 존재한다면 'cuda'라는 값을 할당하고, 존재하지 않는다면 'cpu'라는 값을 할당한다. 차후 각 변수들을 X.to(device)와 같이 설정하여 device로 보낼 수 있다.

### 2. CNN 클래스의 \_\_init\_\_

모델을 설계하는 부분인 CNN 클래스를 유심히 볼 필요가 있다. 먼저 클래스를 정의할 때에는 항상 torch.nn.Module을 상속받아 기능을 모두 사용한다. 또한 layer별로 torch.nn.Sequential 단위로 선언해놓는다. 이는 후에 layer별로 조작을 쉽게 할 수 있다는 장점이 있다.

### 3. 가중치 초기화

CNN 클래스를 살펴보면 `torch.nn.init.xavier_uniform_(self.fc.weight)` 와 같이 FC의 weight의 가중치를 초기화한것을 확인할 수 있다. 즉, 가중치 초기화를 따로 수행함을 알 수 있다.

### 4. FC Layer

Keras는 Flatten() 함수를 통해 자동적으로 feature map을 펼쳐주지만, Pytorch는 `out = out.view(out.size(0), -1)` 와 같이 view 함수와 size 함수의 조합으로 마치 numpy의 reshape처럼 직접 코딩을 통해 펼쳐준다.

### 5. forward(self, x)

CNN 클래스의 마지막 부분은 forward 메서드로 구성되어있다. 이는 모델을 실제로 사용할 때 실행되는 부분으로써, \_\_init\_\_에서 선언한 블록들을 functional API 방식으로 코드화되어있다. 해당 부분을 통해 모델의 흐름을 알 수 있다.

### 6. avg_cost += cost / total_batch

위에서 언급한 바와 같이, Pytorch는 Keras와는 달리 loss값이나 metrics를 직접 정의해서 출력해야한다. avg\_cost 또한 마찬가지로 직접 정의해야 하는데, 위의 코드는 epoch 별 cost값을 total\_batch로 나누어 적용한다. 굳이 이렇게 해야하는지는 의문이다.

### 7. with torch.no_grad():

Pytorch에서 학습을 하지 않고 테스트를 수행할 경우 `with torch.no_grad()`를 통해 학습을 진행하지 않을 것임을 명시해둔다.